# Creating Custom Pipelines

Here we explain how to create custom pipelines in nerfactory. Pipelines are composed of two components, namely a Dataloader and a Model. We'll show how to make an incremental dataloader with scene conditioning.

#TODO(ethan): show a figure depicting the different modules that go into a pipeline

In [3]:
# HIDDEN
%load_ext autoreload
%autoreload 2

from __future__ import annotations

from typing import Dict, Optional, Tuple
import torch

from nerfactory.utils.config import VanillaDataloaderConfig, ModelConfig, PipelineConfig, ViewerConfig

from nerfactory.dataloaders.base import Dataloader
from nerfactory.models.base import Model
from nerfactory.pipelines.base import Pipeline

from nerfactory.models.instant_ngp import NGPModel
from nerfactory.cameras.rays import RayBundle

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### RayBundle class

In the RayBundle class, we allow uses to add non-default attributes for conditioning.


In [ ]:
class IncrementalDataloaderConfig(VanillaDataloaderConfig):
    _target = IncrementalDataloaderConfig
    add_cameras_per_iter: int = 10
    num_cameras_to_add: int = 1

class IncrementalDataloader(Dataloader): # no longer nn.Module
    """A SLAM-based dataloader."""

    config: IncrementalDataloaderConfig

    def populate_train_modules(self):
        """Populate the train dataloader modules."""
        # self.train_image_dataset = ImageDataset # torch dataloader
        # self.train_image_sampler = CacheImageSampler
        # self.train_pixel_sampler = PixelSampler
        # ... 
        # self.train_ray_generator = RayGenerator # nn.Module

    def populate_eval_modules(self):
        """Populate the eval dataloader modules."""
        # self.eval_image_dataset = ImageDataset
        # self.eval_dataloader = FixedIndicesEvalDataloader

    def next_train(self, step: int) -> Tuple[RayBundle, Dict]:
        """Get the next batch of training data by stringing together the train modules."""
        if step % self.config.add_cameras_per_iter == 0:
            print("do something here")
        ray_bundle = RayBundle()
        ray_bundle.metadata["scene_indices"] = None # TODO(ethan): populate this with a tensor of scene indices
        dict_ = {}
        return ray_bundle, dict_

    def next_eval(self, step: int) -> Tuple[RayBundle, Dict]:
        """Get the next batch of eval data by stringing together the eval modules."""
        ray_bundle = RayBundle()
        ray_bundle.metadata["scene_indices"] = None
        dict_ = {}
        return ray_bundle, dict_


class SceneConditionNGPModelConfig(ModelConfig):
    _target =  SceneConditionNGPModel
    coarse_field: str = "temp"
    fine_field: str = "temp2"

class SceneConditionNGPModel(Model):
    """An instant ngp model modified slightly to output semantics."""

    config: SceneConditionNGPModelConfig

    def populate_modules(self):
        self.ngp_model = NGPModel(coarse_field=self.config.coarse_field, fine_field=self.config.fine_field)

    def get_outputs(self, ray_bundle: RayBundle) -> Dict[str, torch.Tensor]:
        # TODO(ethan): pass in batch from the forward function
        # TODO(ethan): rename batch to something else
        outputs = self.ngp_model.forward(ray_bundle)
        outputs["semantics"] = torch.rand_like(outputs["rgb"])
        return outputs

    def get_loss_dict(self):
        return {}

class CustomPipeline(Pipeline):
    """The Instant NGP pipeline."""

    config: PipelineConfig

    def train_step(self, step: int):
        ray_bundle, batch = self.dataloader.next_train(step=step)
        # TODO: maybe run a CNN on the data before passing into model
        model_outputs, loss_dict, metrics_dict = self.model(ray_bundle, batch)
        # TODO: update pixel sampler state with loss map to show the flexibilty of our Pipeline
        return model_outputs, loss_dict, metrics_dict

    def eval_step(self):
        raise NotImplementedError


In [ ]:
dataloader = Dataloader()
model = SceneConditionNGPModel()
pipeline = CustomPipeline.from_dataloader_and_model(dataloader=dataloader, model=model)

# Creating pipelines from a config

Now we show how to create a pipeline from a config, which has the following form:

```python
@dataclass
class PipelineConfig:
    """Configuration for pipeline instantiation"""

    _target_: str = MISSING
    dataloader_config: DataloaderConfig = MISSING
    graph_config: GraphConfig = MISSING
```

See [nerfactory/utils/config.py](nerfactory/utils/config.py) for more details. In this example, we will simply load from an existing configuration from [configs/graph_instant_ngp.yaml](configs/graph_instant_ngp.yaml).

In [ ]:
import pprint
import hydra

hydra.core.global_hydra.GlobalHydra.instance().clear()
from hydra import compose, initialize

initialize(version_base="1.2", config_path="../configs/")
config_name = "graph_instant_ngp.yaml"
config = compose(config_name)
pipeline_config = config.pipeline
pprint.pprint(pipeline_config)
print("----------------------------------------------------")

from nerfactory.pipelines.base import setup_pipeline

pipeline = setup_pipeline(pipeline_config, device="cuda")

In [ ]:
pipeline.get_train_loss_dict()